In [13]:
import os
from datetime import date, datetime, timedelta

import pandas as pd

In [ ]:
# T_將raw data讀入

folder = r"C:\Users\Bevis\Documents\Python_project\sample\output\gmap_search"
file_TPE = f"gmap_lodging_TPE.csv"
file_TYU = f"gmap_lodging_TYU.csv"
file_TCH = f"gmap_lodging_TCH.csv"
file_TNA = f"gmap_lodging_TNA.csv"
file_KSH = f"gmap_lodging_KSH.csv"


path_TPE = os.path.join(folder, file_TPE)
path_TYU = os.path.join(folder, file_TYU)
path_TCH = os.path.join(folder, file_TCH)
path_TNA = os.path.join(folder, file_TNA)
path_KSH = os.path.join(folder, file_KSH)


df_TPE = pd.read_csv(path_TPE)
df_TYU = pd.read_csv(path_TYU)
df_TCH = pd.read_csv(path_TCH)
df_TNA = pd.read_csv(path_TNA)
df_KSH = pd.read_csv(path_KSH)

df_dict = {
    "TPE" : df_TPE,
    "TYU" : df_TYU,
    "TCH" : df_TCH,
    "TNA" : df_TNA,
    "KSH" : df_KSH
    }

city_list = list(df_dict.keys())

In [ ]:
# T_清理、去重複、重組資料

columns = ['name', 'place_id', 'buss_status', 'update_time']
main_df = pd.DataFrame(columns=columns)

for city in city_list:
    df_edit = df_dict[city].copy()

    # 第一步篩選：將營業狀態非「正常營業」的地標去除
    mask1 = df_edit["buss_status"] == "OPERATIONAL"
    df_edit = df_edit[mask1]

    # 第二部篩選：根據place id將重複的資料去除
    df_edit = df_edit.drop_duplicates(subset=["place_id"], keep="first")
    print(f"完成{city}資料處理，共{len(df_edit)}筆資料")

    # 將城市資料合併至主要dataframe
    main_df = pd.concat([main_df, df_edit])

# 合併完成後再去重複一次
main_df = main_df.drop_duplicates(subset=["place_id"], keep="first")

# 重設索引
main_df = main_df.reset_index(drop=True)

# 新增id欄位並改變位置至最前
main_df["id"] = "lodging_raw00" + main_df.index.astype(str)
new_col = ["id", "name", "buss_status", "place_id"]
main_df = main_df[new_col]
print(f"完成所有資料整合，共{len(main_df)}筆資料")

# 標記更新日期
today = date.today().strftime('%Y/%m/%d')
main_df["update_date"] = today

完成TPE資料處理，共318筆資料
完成TYU資料處理，共179筆資料
完成TCH資料處理，共223筆資料
完成TNA資料處理，共137筆資料
完成KSH資料處理，共169筆資料


In [ ]:
# T_存檔為csv
file_main = "gmap_lodging_main_placeid.csv"
path_main = os.path.join(folder, file_main)

main_df.to_csv(path_main, index=False)

In [ ]:
# T_存檔至資料庫
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

main_df.to_sql(name="lodging_placeid", con=engine, if_exists="replace", index=False)

978